In [1]:
import sys
sys.path.append('../build')   # assume an out-of-source build
import exafmm_laplace as fmm
import numpy as np

In [2]:
fmm.__doc__

"exafmm's pybind11 module for Laplace kernel"

#### create sources and targets

In [3]:
nsrcs = 100000
ntrgs = 100000

In [4]:
src_coords = np.random.random((nsrcs, 3))
src_charges = np.random.random(nsrcs)
trg_coords = np.random.random((ntrgs, 3))

In [5]:
sources = fmm.init_sources(src_coords, src_charges)
targets = fmm.init_targets(trg_coords)

#### set FMM parameters

In [6]:
p = 10       # expansion order
ncrit = 100  # max number of bodies allow per leaf box
fmm.configure(p, ncrit)

#### build tree

In [7]:
nodes = fmm.build_tree(sources, targets)

#### build interaction lists

In [8]:
fmm.build_list()

#### precompute

In [9]:
fmm.precompute()

#### evaluate potentials

In [10]:
nodes = fmm.evaluate()

P2M                  : 1.8133100e-01
M2M                  : 1.1962900e-01
P2L                  : 1.5500000e-03
M2P                  : 1.5030000e-03
P2P                  : 4.6648000e-02
M2L                  : 6.7533500e-01
L2L                  : 8.5321000e-02
L2P                  : 1.3302300e-01


#### check accuracy

In [11]:
fmm.check_accuracy()

-------------- Error ---------------
Potential Error      : 1.7633568e-09
Gradient Error       : 1.6858267e-07
